In [1]:
import pandas as pd
from os import path, environ, makedirs
import requests as re
from io import BytesIO
from dotenv import load_dotenv
from unidecode import unidecode
import numpy as np

In [2]:
load_dotenv()

True

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `05 ObservaSampa - extração.ipynb`.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

In [3]:
output_dir = path.join('dados', 'urbanismo')
if not path.exists(output_dir):
    makedirs(output_dir)

In [4]:
indicador_domicilios_nao_ocupados_df = pd.read_csv(path.join(output_dir, 'indicador_domicilios_nao_ocupados.csv'),
                        sep=';', decimal=',', encoding='utf8', dtype=str)

In [5]:
indicador_domicilios_nao_ocupados_df.dtypes

cd_subprefeitura                        object
nm_subprefeitura                        object
domicilios_particulares_nao_ocupados    object
total_domicilios_particulares           object
domicilios_particulares_ocupados        object
dtype: object

## Indicador domicilios não ocupados (censo 2022)

In [6]:
df = indicador_domicilios_nao_ocupados_df.copy()

In [7]:
df["cd_subprefeitura"] = df["cd_subprefeitura"].astype(str).str.zfill(2)
df["nm_subprefeitura"] = df["nm_subprefeitura"].astype(str).str.strip()

In [8]:
num_cols = [
    "domicilios_particulares_nao_ocupados",
    "total_domicilios_particulares",
    "domicilios_particulares_ocupados",
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["tx_nao_ocupados_pct"] = (
    np.where(df["total_domicilios_particulares"].gt(0),
             df["domicilios_particulares_nao_ocupados"] / df["total_domicilios_particulares"] * 100,
             np.nan)
)

In [9]:
cols_final = [
    "cd_subprefeitura",
    "nm_subprefeitura",
    "domicilios_particulares_nao_ocupados",
    "domicilios_particulares_ocupados",
    "total_domicilios_particulares",
    "tx_nao_ocupados_pct",
]

In [10]:
df = df[cols_final].sort_values(["cd_subprefeitura"]).reset_index(drop=True)

In [11]:
df_long = df[["cd_subprefeitura", "nm_subprefeitura", "domicilios_particulares_nao_ocupados"]].rename(
    columns={"domicilios_particulares_nao_ocupados": "valor"}
)

In [12]:
df_long["id_observa"] = 999 
df_long = df_long[["id_observa", "cd_subprefeitura", "nm_subprefeitura", "valor"]]

In [13]:
df_domicilios_nao_ocupados_tratado = df
df_domicilios_nao_ocupados_long = df_long

In [14]:
df

,cd_subprefeitura,nm_subprefeitura,domicilios_particulares_nao_ocupados,domicilios_particulares_ocupados,total_domicilios_particulares,tx_nao_ocupados_pct
0,01,PERUS,7826,56874,64700,12.095827
1,02,PIRITUBA-JARAGUA,25143,175623,200766,12.523535
2,03,FREGUESIA-BRASILANDIA,19562,138060,157622,12.410704
3,04,CASA VERDE-CACHOEIRINHA,16151,113822,129973,12.426427
4,05,SANTANA-TUCURUVI,22676,125151,147827,15.339552
5,06,JACANA-TREMEMBE,13860,99015,112875,12.279070
6,07,VILA MARIA-VILA GUILHERME,15956,104177,120133,13.281946
7,08,LAPA,21194,141589,162783,13.019787
8,09,SE,54320,202905,257225,21.117699
9,10,BUTANTA,25508,178454,203962,12.506251


In [15]:
df_long

,id_observa,cd_subprefeitura,nm_subprefeitura,valor
0,999,01,PERUS,7826
1,999,02,PIRITUBA-JARAGUA,25143
2,999,03,FREGUESIA-BRASILANDIA,19562
3,999,04,CASA VERDE-CACHOEIRINHA,16151
4,999,05,SANTANA-TUCURUVI,22676
5,999,06,JACANA-TREMEMBE,13860
6,999,07,VILA MARIA-VILA GUILHERME,15956
7,999,08,LAPA,21194
8,999,09,SE,54320
9,999,10,BUTANTA,25508


# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [16]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [
                 ('indicador_domicilios_nao_ocupados', df),
                 ('indicador_domicilios_nao_ocupados_long', df_long),
                 ]:
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')